In [ ]:
!which python

/usr/local/bin/python


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers==4.40.1
!pip install av
!pip install imageio
!pip install decord
!pip install opencv-python

In [ ]:
'''# prompt: open file /usr/local/lib/python3.11/dist-packages/transformers/feature_extraction_utils.py and change this line: return torch.tensor(value) with this: return torch.tensor(value, dtype=torch.float16)  check to see if changes are made

# Read the content of the file
with open('/usr/local/lib/python3.11/dist-packages/transformers/feature_extraction_utils.py', 'r') as file:
  file_content = file.read()
# Replace the line
file_content_modified = file_content.replace(
    'return torch.tensor(value, dtype=torch.float16)',
    'return torch.tensor(value)'

)
# Write the modified content back to the file
with open('/usr/local/lib/python3.11/dist-packages/transformers/feature_extraction_utils.py', 'w') as file:
  file.write(file_content_modified)
# Verify the change
with open('/usr/local/lib/python3.11/dist-packages/transformers/feature_extraction_utils.py', 'r') as file:
  file_content_check = file.read()
if 'return torch.tensor(value, dtype=torch.float16)' in file_content_check:
  print("Change successfully applied.")
else:
  print("Change not found in the file.")'''

In [ ]:
'''# change the line: import torchvision.transforms.functional_tensor as F_t to import torchvision.transforms.functional as F_t and check to see if changes are made

filepath = '/usr/local/lib/python3.11/dist-packages/pytorchvideo/transforms/augmentations.py'

# Read the content of the file
with open(filepath, 'r') as file:
    file_content = file.read()

# Replace the specific line
old_line = 'import torchvision.transforms.functional_tensor as F_t'
new_line = 'import torchvision.transforms.functional as F_t'
file_content = file_content.replace(old_line, new_line)

# Write the modified content back to the file
with open(filepath, 'w') as file:
    file.write(file_content)

# Check if the changes were made by reading the file again and searching for the new line
with open(filepath, 'r') as file:
    updated_content = file.read()

if new_line in updated_content:
    print(f"Successfully changed '{old_line}' to '{new_line}' in {filepath}")
else:
    print(f"Failed to change '{old_line}' to '{new_line}' in {filepath}")
'''

In [ ]:
import json
import os
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer


In [ ]:
json_filename = "/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/BEST QUERIES/inputs/labeled_top_50_queries.json"
clip_folder = "/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/inputs/re_clips"
output_json = "/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/outputs/InternVideo2_5_labeled_top_50_results.json"

In [ ]:
# model setting
model_path = 'OpenGVLab/InternVideo2_5_Chat_8B'

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda().to(torch.bfloat16)

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


FlashAttention2 is not installed.


model.safetensors.index.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model-00004-of-00004.safetensors:   0%|          | 0.00/2.03G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([T.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img), T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC), T.ToTensor(), T.Normalize(mean=MEAN, std=STD)])
    return transform

In [ ]:
def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float("inf")
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

In [ ]:
def dynamic_preprocess(image, min_num=1, max_num=6, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set((i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = ((i % (target_width // image_size)) * image_size, (i // (target_width // image_size)) * image_size, ((i % (target_width // image_size)) + 1) * image_size, ((i // (target_width // image_size)) + 1) * image_size)
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

In [ ]:
def load_image(image, input_size=448, max_num=6):
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [ ]:
def get_index(bound, fps, max_frame, first_idx=0, num_segments=32):
    if bound:
        start, end = bound[0], bound[1]
    else:
        start, end = -100000, 100000
    start_idx = max(first_idx, round(start * fps))
    end_idx = min(round(end * fps), max_frame)
    seg_size = float(end_idx - start_idx) / num_segments
    frame_indices = np.array([int(start_idx + (seg_size / 2) + np.round(seg_size * idx)) for idx in range(num_segments)])
    return frame_indices

In [ ]:
def get_num_frames_by_duration(duration):
        local_num_frames = 4
        num_segments = int(duration // local_num_frames)
        if num_segments == 0:
            num_frames = local_num_frames
        else:
            num_frames = local_num_frames * num_segments

        num_frames = min(512, num_frames)
        num_frames = max(128, num_frames)

        return num_frames

In [ ]:
def load_video(video_path, bound=None, input_size=448, max_num=1, num_segments=32, get_frame_by_duration = False):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr) - 1
    fps = float(vr.get_avg_fps())

    pixel_values_list, num_patches_list = [], []
    transform = build_transform(input_size=input_size)
    if get_frame_by_duration:
        duration = max_frame / fps
        num_segments = get_num_frames_by_duration(duration)
    frame_indices = get_index(bound, fps, max_frame, first_idx=0, num_segments=num_segments)
    for frame_index in frame_indices:
        img = Image.fromarray(vr[frame_index].asnumpy()).convert("RGB")
        img = dynamic_preprocess(img, image_size=input_size, use_thumbnail=True, max_num=max_num)
        pixel_values = [transform(tile) for tile in img]
        pixel_values = torch.stack(pixel_values)
        num_patches_list.append(pixel_values.shape[0])
        pixel_values_list.append(pixel_values)
    pixel_values = torch.cat(pixel_values_list)
    return pixel_values, num_patches_list


In [ ]:
def predict(qs, video_path):
    # evaluation setting
    max_num_frames = 512
    generation_config = dict(
        do_sample=False,
        temperature=0.0,
        max_new_tokens=1024,
        top_p=0.1,
        num_beams=1
    )

    num_segments = 128  # Make sure this is kept in sync

    with torch.no_grad():
        pixel_values, num_patches_list = load_video(video_path, num_segments=num_segments, max_num=1, get_frame_by_duration=False)
        pixel_values = pixel_values.to(torch.bfloat16).to(model.device)

        # Add one <image> token per patch
        num_images = pixel_values.shape[0]
        image_tokens = ''.join(['<image>\n' for _ in range(num_images)])
        full_prompt = f"{image_tokens}{qs}"

        response, chat_history = model.chat(
            tokenizer,
            pixel_values,
            full_prompt,
            generation_config,
            num_patches_list=num_patches_list,
            history=None,
            return_history=True
        )

    return response


In [ ]:
# === PROCESS ALL CLIPS ===
results = []
with open(json_filename, 'r') as f:
    nlq_entries = json.load(f)

for entry in nlq_entries:
    clip_uid = entry["clip_uid"]
    query_idx = entry["query_idx"]
    question = entry["question"]
    gt_answer = entry["answer"]
    video_path = os.path.join(clip_folder, f"{clip_uid}_{query_idx}.mp4")
    print(video_path)


    model_answer = predict(question, video_path)

    results.append({
        "clip_uid": clip_uid,
        "query_idx": query_idx,
        "question": question,
        "gt_answer": gt_answer,
        "model_answer": model_answer
    })
    print(f"✅ {clip_uid}_{query_idx}")


/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/re_clips/8d3f5b12-ac2c-4315-80ed-bb827aa91bd4_0.mp4


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


✅ 8d3f5b12-ac2c-4315-80ed-bb827aa91bd4_0
/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/re_clips/c664f078-9b34-4a58-b949-180ac4bc0980_4.mp4


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


✅ c664f078-9b34-4a58-b949-180ac4bc0980_4
/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/re_clips/c7873f07-7b9c-4794-9a56-fdebeeceee68_6.mp4


...

✅ 47c24350-cc10-4ec6-8407-24f778981a82_0
/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/re_clips/ed0a0e94-c79b-462d-a64b-238f26fd6fc6_6.mp4


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


✅ ed0a0e94-c79b-462d-a64b-238f26fd6fc6_6
/content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/re_clips/b1bd830a-a738-4ebe-9e88-c348def0de90_4.mp4


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


✅ b1bd830a-a738-4ebe-9e88-c348def0de90_4


In [ ]:
# === SAVE OUTPUT ===
with open(output_json, "w") as f:
    json.dump(results, f, indent=2)

print(f"🎉 Done. Results saved to: {output_json}")

🎉 Done. Results saved to: /content/drive/MyDrive/Colab Notebooks/Egocentric Vision/Video_Query_to_Answer/outputs/labeled_top_50_results.json
